In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [9]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [12]:
shooting.head()

For Arsenal                                                              \
         Date   Time            Comp        Round  Day Venue Result GF GA   
0  2022-08-05  20:00  Premier League  Matchweek 1  Fri  Away      W  2  0   
1  2022-08-13  15:00  Premier League  Matchweek 2  Sat  Home      W  4  2   
2  2022-08-20  17:30  Premier League  Matchweek 3  Sat  Away      W  3  0   
3  2022-08-27  17:30  Premier League  Matchweek 4  Sat  Home      W  2  1   
4  2022-08-31  19:30  Premier League  Matchweek 5  Wed  Home      W  2  1   

                   ... Standard               Expected                    \
         Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0  Crystal Palace  ...     14.6  1.0  0     0      1.0  1.0    0.10  0.0   
1  Leicester City  ...     13.0  0.0  0     0      2.7  2.7    0.16  1.3   
2     Bournemouth  ...     14.8  0.0  0     0      1.3  1.3    0.10  1.7   
3          Fulham  ...     15.5  1.0  0     0      2.6  2.6    0.12 -0.6   
4     Aston Villa  ...     16.3  1.0  0     0      2.4  2.4    0.12 -0.4   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0     0.0        Match Report  
1     1.3        Match Report  
2     1.7        Match Report  
3    -0.6        Match Report  
4    -0.4        Match Report  

[5 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel()

In [14]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [15]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,4-3-3,Anthony Taylor,Match Report,NaN,10,2,14.6,1.0,0,0
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,4-2-3-1,Darren England,Match Report,NaN,19,7,13.0,0.0,0,0
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,4-2-3-1,Craig Pawson,Match Report,NaN,14,6,14.8,0.0,0,0
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,4-2-3-1,Jarred Gillett,Match Report,NaN,22,8,15.5,1.0,0,0
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,4-2-3-1,Robert Jones,Match Report,NaN,22,8,16.3,1.0,0,0


In [16]:
years = list(range(2022, 2020, -1))
all_matches = []

In [17]:
years

[2022, 2021]

In [18]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [19]:
import time
for year in years:
    time.sleep(1)
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    time.sleep(1)
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    time.sleep(1)
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        time.sleep(1)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        time.sleep(1)
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

# 3min

In [20]:
len(all_matches)

40

In [21]:
match_df = pd.concat(all_matches)

In [22]:
match_df.columns = [c.lower() for c in match_df.columns]

In [23]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,14.6,1.0,0.0,0.0,2022,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,13.0,0.0,0.0,0.0,2022,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.8,0.0,0.0,0.0,2022,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,22.0,8.0,15.5,1.0,0.0,0.0,2022,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Match Report,NaN,22.0,8.0,16.3,1.0,0.0,0.0,2022,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2022-04-30,15:00,Premier League,Matchweek 35,Sat,Away,L,0,2,Aston Villa,...,Match Report,NaN,9.0,3.0,21.6,0.0,0.0,0.0,2021,Norwich City
39,2022-05-08,14:00,Premier League,Matchweek 36,Sun,Home,L,0,4,West Ham,...,Match Report,NaN,8.0,2.0,22.2,1.0,0.0,0.0,2021,Norwich City
40,2022-05-11,19:45,Premier League,Matchweek 21,Wed,Away,L,0,3,Leicester City,...,Match Report,NaN,9.0,5.0,17.0,0.0,0.0,0.0,2021,Norwich City
41,2022-05-15,14:00,Premier League,Matchweek 37,Sun,Away,D,1,1,Wolves,...,Match Report,NaN,11.0,2.0,14.4,0.0,0.0,0.0,2021,Norwich City
